<a href="https://colab.research.google.com/github/AshwinDeshpande96/Hierarchical-Softmax/blob/master/CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

169017344/169001437 [==============================] - 19s 0us/step


In [0]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, Layer
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
import tensorflow as tf
import re
import time as t
import itertools as it
from random import shuffle
from copy import copy


K.set_image_dim_ordering('th')

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
x_train = x_train.astype('float32').reshape((x_train.shape[0], 3, 32, 32))/255.
x_test = x_test.astype('float32').reshape((x_test.shape[0], 3, 32, 32))/255.
#y_train = -K.log(np_utils.to_categorical(y_train))
#y_test = -K.log(np_utils.to_categorical(y_test))
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
print x_train.shape, y_train.shape

(50000, 3, 32, 32) (50000, 100)


In [0]:

class TreeTools:
    def __init__(self):
        #memoization for _count_nodes functions
        self._count_nodes_dict = {}
        self.index = 0
    def _get_paths(self, tree, i=0, i_list=[]):
        key = ','.join(str(x) for x in i_list[:])
        yield key, i_list
        self.index = self.index + 1
                   
        for subtree in tree:
            if type(subtree) == list:
                for node, ilist in self._get_paths(subtree, i=self.index, i_list=i_list[:]+[self.index]):
                    if (type(subtree[0]) == int):
                        node = "e".join(str(n) for n in subtree)
                    
                    yield node, ilist
            else:
                if type(tree[0]) == list:
                    yield "e"+str(subtree), i_list            
    def _get_subtrees(self, tree):
        yield tree
        for subtree in tree:
            if type(subtree) == list:
                for x in self._get_subtrees(subtree):
                    yield x

    # Returns pairs of paths and leafves of a tree
    def _get_leaves_paths(self, tree, i_list=[]):
        for i, subtree in enumerate(tree):
            if type(subtree) == list:
                for path, value in self._get_leaves_paths(subtree):
                    yield [i] + path, value
            else:
                yield [i], subtree
    
    # Returns the number of nodes in a tree (not including root)
    def _count_nodes(self, tree):
        if id(tree) in self._count_nodes_dict:
            return self._count_nodes_dict[id(tree)]
        size = 0
        for node in tree:
            if type(node) == list:
                size += 1 + self._count_nodes(node)
        self._count_nodes_dict[id(self._count_nodes_dict)] = size
        return size


    # Returns all the nodes in a path
    def _get_nodes(self, tree, path):
        next_node = 0
        nodes = []
        for decision in path:
            nodes.append(next_node)
            next_node += 1 + self._count_nodes(tree[:decision])
            tree = tree[decision]
        return nodes
    def _print_tree(tree):
        print tree
        for subtree in tree:
            self._print_tree(subtree)

# turns a list to a binary tree
def make_tree(output_dim):
    outputs = list(range(output_dim))
    outputs = copy(outputs)
    shuffle(outputs)
    
    while len(outputs) > 2:
        temp_outputs = []
        for i in range(0, len(outputs), 2):
            if len(outputs) - (i+1) > 0:
                temp_outputs.append([outputs[i], outputs[i+1]])
            else:
                temp_outputs.append(outputs[i])
        outputs = temp_outputs
    return outputs

In [0]:
tree = random_binary_full_tree(list(range(10)))
treetool = TreeTools()
for ilist, node in treetool._get_paths(tree):
    
    print ilist


1
1,2
8e4
6e7
1,5
2e0
3e1
9e5


In [0]:
from keras import backend as K
from keras.layers import Layer
import tensorflow as tf
import re

class HSM(Layer):

    def __init__(self, output_dim, **kwargs):
        super(HSM, self).__init__(**kwargs)
        
        self.output_dim = output_dim
        self.tree = make_tree(output_dim)
        '''
        self.node_vectors = {}
        
        for i  in range(1, self.output_dim):
            self.node_vectors[i] = self.add_weight(name="node_"+str(i),
                                                   shape=(len(self.tree), len(self.tree)),
                                                   initializer='uniform',
                                                   trainable=True)
        '''
        
    def build(self, input_shape):
        
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(self.output_dim, input_shape[1], len(self.tree)),
                                      initializer='uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='bias', 
                                      shape=(self.output_dim,),
                                      initializer='zeros',
                                      trainable=True)
        
        super(HSM, self).build(input_shape)  # Be sure to call this at the end
             
    def call(self, x):
        x = K.dot(x, self.kernel)
        print x
        
        return x

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [0]:
sess = tf.Session()
x = tf.convert_to_tensor([i for i in range(12)], preferred_dtype=tf.dtypes.float32)
w = tf.convert_to_tensor(np.ones((10,12,2)), preferred_dtype=tf.dtypes.float32)
print sess.run(K.dot(x,w))

[[66. 66.]
 [66. 66.]
 [66. 66.]
 [66. 66.]
 [66. 66.]
 [66. 66.]
 [66. 66.]
 [66. 66.]
 [66. 66.]
 [66. 66.]]


In [0]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(3, 32, 32), padding='same', activation='relu', name='conv1'))
#model.add(BatchNormalization( name='bn1'))
model.add(MaxPooling2D(pool_size=(17, 17) , name='max1'))
#model.add(Conv2D(16, (5, 5), activation='relu', name='conv2'))
#model.add(BatchNormalization( name='bn2'))
#model.add(MaxPooling2D(pool_size=(2, 2), name='max2'))
#model.add(Dropout(0.2, name='drop1'))
model.add(Flatten(name='flat1'))
model.add(Dense(128, name='dense1'))
#model.add(Dense(output_dim=num_classes, activation='softmax', name='dense2'))
hsm1 = HSM(output_dim=100, name='hsm1')
model.add(hsm1)
#model.add(Activation('softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(1.0e-3), metrics=['accuracy'])
model.summary()
#print model.trainable_weights
#print model.non_trainable_weights

Tensor("hsm1_15/Reshape_2:0", shape=(?, 100, 2), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 32, 32, 32)        2432      
_________________________________________________________________
max1 (MaxPooling2D)          (None, 32, 1, 1)          0         
_________________________________________________________________
flat1 (Flatten)              (None, 32)                0         
_________________________________________________________________
dense1 (Dense)               (None, 128)               4224      
_________________________________________________________________
hsm1 (HSM)                   (None, 100)               25700     
Total params: 32,356
Trainable params: 32,356
Non-trainable params: 0
_________________________________________________________________


In [0]:
# build the model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=1.5e-5, patience=5, verbose=1, mode='min')
    #ModelCheckpoint(filepath, save_best_only=True,  save_weights_only=False, monitor='val_loss', mode='min', verbose=1)
]
# Fit the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=callbacks, epochs=50, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
 - 3s - loss: 4.3655 - acc: 0.0420 - val_loss: 4.1263 - val_acc: 0.0736
Epoch 2/50
 - 2s - loss: 3.9977 - acc: 0.0934 - val_loss: 3.9185 - val_acc: 0.1083
Epoch 3/50
 - 2s - loss: 3.8272 - acc: 0.1210 - val_loss: 3.7888 - val_acc: 0.1287
Epoch 4/50
 - 2s - loss: 3.7274 - acc: 0.1358 - val_loss: 3.7219 - val_acc: 0.1390
Epoch 5/50
 - 2s - loss: 3.6666 - acc: 0.1473 - val_loss: 3.6704 - val_acc: 0.1489
Epoch 6/50
 - 2s - loss: 3.6201 - acc: 0.1558 - val_loss: 3.6518 - val_acc: 0.1546
Epoch 7/50


KeyboardInterrupt: ignored

# rough

In [0]:

number_of_leaves = 100
tree1 = random_binary_full_tree(list(range(number_of_leaves)))
tt = TreeTools()
root = tf.convert_to_tensor(np.random.rand(number_of_leaves,2).astype('float32'))
node_vectors = {0: root}
saved_weights = {}
x = tf.convert_to_tensor(np.random.rand(1, number_of_leaves).astype('float32'))


for i  in range(1, number_of_leaves):
    node_vectors[i] = tf.convert_to_tensor(np.random.uniform(0.0,1.0, 4).reshape((2,2)), preferred_dtype='float32')


def node_prob(x, path):
    key = ",".join(str(n) for n in path)
    if key in saved_weights.keys():
        #print "Found: ", key
        return saved_weights[key]
    else:
        if len(path) == 1:
            #print "Compute: ", x, "*", node_vectors[path[0]] 
            return K.softmax(K.dot(x, node_vectors[path[0]]))
        a = node_prob(x, path[:-1])
        b = node_vectors[path[-1]]
        #print "Compute: ", a, "*", b
        return K.softmax(K.dot(a, b))

sess = tf.Session()
leaf_to_decision = {}
for path, value in tt._get_leaves_paths(tree1):
    leaf_to_decision[value] = path

def get_marginal_prob(path, decision):
    prob = tf.zeros([1,1])
    #prob = tf.ones([1,1])
    for i in range(0,len(path)):
        key = ",".join(str(node) for node in path[:i+1])
        d = decision[i]
        p = tf.slice(saved_weights[key], [0,d], [1,1])
        l = tf.log(p)
        #print d, path[i], sess.run(saved_weights[key]), sess.run(p), sess.run(l)
        prob = prob + -1*(l)
        #prob *= p
    #print sess.run(prob)
    return prob
output = [tf.zeros([1,1], dtype=tf.dtypes.float32) for repeater in range(number_of_leaves)]
#output = []
start = t.time()
for node, path in tt._get_paths(tree1):
    #print node, "\t", path
    
    key = ",".join(str(p) for p in path)
    temp = node_prob(x, path)
    
    #print temp.get_shape(), "\t", node, "\t\t", path
    saved_weights[key] = temp
    
    if 'e' in node:
        if node.startswith('e'):
            last_leaf = int(node[1:])
            probability = get_marginal_prob(path, leaf_to_decision[last_leaf])
            #print "p(", last_leaf, ")= ", sess.run(probability)
            #output.append(probability )
            output[last_leaf] =  probability
            continue
        end_node = [int(leaf) for leaf in node.split('e')]
        for leaves in end_node:
            #print ""
            probability = get_marginal_prob(path, leaf_to_decision[leaves])
            #print "p(", leaves, ")= ", sess.run(probability)
            #output.append(probability )
            output[leaves] =  probability
    #print ""
tt.index = 0      
time_taken = t.time() - start
output = tf.concat(output, axis=1)

print "Time Taken: ", time_taken, "s\nNumber of Classes: ", number_of_leaves
#print sess.run(output)



Time Taken:  2.43883395195 s
Number of Classes:  100


In [0]:

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        
        self.output_dim = output_dim
        self._tree_tools = TreeTools()
        self.tree = random_binary_full_tree(list(range(output_dim)))
        #print self.tree
        self.saved_weights = {}
        self.leaf_to_decision = {}
        for path, value in self._tree_tools._get_leaves_paths(self.tree):
            self.leaf_to_decision[value] = path
        '''
        leaf_to_path = {}
        for path, value in self._tree_tools._get_leaves_paths(tree):
            nodes = self._tree_tools._get_nodes(tree, path)
            leaf_to_path[int(value)] = path, nodes
        self.leaf_to_path = leaf_to_path
        
        self.str2weight = {}
        for i, subtree in enumerate(self._tree_tools._get_subtrees(tree)):
            self.str2weight["softmax_node_"+str(i)+"_w"] = self.add_weight(name="softmax_node_"+str(i)+"_w",
                                                                          shape=(len(subtree), output_dim),
                                                                          initializer='uniform',
                                                                          trainable=True)
            self.str2weight["softmax_node_" + str(i) + "_b"] = self.add_weight(name="softmax_node_"+str(i)+"_b",
                                                                          shape=(len(subtree),1),
                                                                          initializer='zeros',
                                                                          trainable=True)
        '''
        #print "Length of the first layer:", len(self.tree)
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        self.bias = self.add_weight(name='bias', 
                                      shape=(self.output_dim,),
                                      initializer='uniform',
                                      trainable=True)

        self.node_vectors = {0: self.add_weight(name="root_node",
                                                   shape=(self.output_dim, len(self.tree)),
                                                   initializer='uniform',
                                                   trainable=True)}
        
        for i  in range(1, self.output_dim):
            self.node_vectors[i] = self.add_weight(name="node_"+str(i),
                                                   shape=(len(self.tree), len(self.tree)),
                                                   initializer='uniform',
                                                   trainable=True)
        
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end
    def get_marginal_prob(self, path, decision):
        #prob = tf.zeros([1,1])
        prob = tf.ones([1,1])
        for i in range(0,len(path)):
            key = ",".join(str(node) for node in path[:i+1])
            d = decision[i]
            p = tf.slice(self.saved_weights[key], [0,d], [1,1])
            #l = tf.log(p)
            #prob = prob + -1*(l)
            prob *= p
            
            #print d, path[i], sess.run(self.saved_weights[key]), sess.run(p), sess.run(l)
        #print sess.run(prob)
        return prob
    def node_prob(self, x, path):
        key = ",".join(str(n) for n in path)
        if key in self.saved_weights.keys():
            #print "Found: ", key, "\n\t",
            return self.saved_weights[key]
        else:
            if len(path) == 1:
                
                #print "Compute: ", x, "*", self.node_vectors[path[0]]
                
                return K.softmax(K.dot(x, self.node_vectors[path[0]]))
            a = self.node_prob(x, path[:-1])
            b = self.node_vectors[path[-1]]
            #print "Compute: ", a, "*", b
            return K.softmax(K.dot(a, b))
                
    def call(self, x):
        x = K.dot(x, self.kernel)
        x = x + self.bias
        #self.node_vectors = {0: K.softmax(K.dot(x, self.root))}
        
        '''
        
        output = [tf.zeros([1,1], dtype=tf.dtypes.float32) for repeater in range(self.output_dim)]
        self.saved_weights = {}
        for node, path in self._tree_tools._get_paths(self.tree):
            key = ",".join(str(p) for p in path)

            #print node, "\t", path
            self.saved_weights[key] = self.node_prob(x, path)
            if 'e' in node:
                if node.startswith('e'):
                    last_leaf = int(node[1:])
                    probability = self.get_marginal_prob(path, self.leaf_to_decision[last_leaf])
                    #print "p(", last_leaf, ")= ", sess.run(probability)
                    #output.append(probability )
                    output[last_leaf] =  probability
                    continue
                end_node = [int(leaf) for leaf in node.split('e')]
                for leaves in end_node:
                    #print ""
                    probability = self.get_marginal_prob(path, self.leaf_to_decision[leaves])
                    #print "p(", leaves, ")= ", sess.run(probability)
                    #output.append(probability )
                    output[leaves] =  probability
        self._tree_tools.index = 0
        output = tf.concat(output, axis=1)
        
        return output
        
        '''
        '''
        #loss = tf.convert_to_tensor(np.random.rand(100,).astype('float32'))
        
        loss = []
        
        for value in self.leaf_to_path:
            log_loss = tf.convert_to_tensor([0.0])
            path, node = self.leaf_to_path[value]
            #print "\n"
            #print path, node
            for i in range(len(path)):
                p = path[i]
                n = node[i]
                #print p,n
                vec = self.str2weight["softmax_node_"+str(n)+"_w"]
                b = self.str2weight["softmax_node_" + str(n) + "_b"]
                
                
                preds = softmax(vec,x,b)
                node_loss = -1 * K.log(tf.slice(preds, [p,0], [1,1]))
                log_loss = log_loss + node_loss
                
            loss.append(log_loss)  
                
        
        return tf.reshape(loss, [100,1])
        '''
        return K.softmax(x)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [0]:

def softmax2(vec, x1, b):
    """Softmax activation function.
    # Arguments
        x: Input tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    vec = tf.convert_to_tensor(vec)
    x1 = tf.reshape(x1, [128,1])
    #x1 = tf.convert_to_tensor(x1)
    x = K.dot(vec, x1)
    x = tf.math.add(x, b)
    
    e = K.exp(x)
    s = K.sum(e)
    #sess = tf.Session()
    
    return e/s
#get_custom_objects().update({'custom_activation': Activation(softmax2)})


In [0]:

w1 = tf.convert_to_tensor([[20,2,3], [1,2,2]], preferred_dtype='float32')
x1 = tf.convert_to_tensor([[1],[1],[1]], preferred_dtype='float32')
b1 = tf.convert_to_tensor([[1],[2]], preferred_dtype='float32')
res = K.dot(w1/20, x1)
x = tf.math.add(res, b1/2)/20
e = K.exp(x)
s = K.sum(e)
val = tf.slice(b1, [1,0], [1,1])
sess = tf.Session()
#print sess.run(val)
var = tf.convert_to_tensor([0.0])

var = var + val
l = []
print sess.run(var)
l.append(var)
var = var + val
print sess.run(var)
l.append(var)
print 

In [0]:
tree = random_binary_full_tree(list(range(10)))
tree_tools = TreeTools()
leaf_to_path = {}
for path, value in tree_tools._get_leaves_paths(tree):
    nodes = tree_tools._get_nodes(tree, path)
    leaf_to_path[int(value)] = path, nodes
str2weight = {}
for i, subtree in enumerate(tree_tools._get_subtrees(tree)):
    str2weight["softmax_node_"+str(i)+"_w"] = np.random.rand(len(subtree),128)
    str2weight["softmax_node_"+str(i)+"_b"] = np.random.rand(1, 128)


In [0]:
for value in leaf_to_path:
    print value
    path, node = leaf_to_path[value]
    node_probs = []
    for p, n in zip(path, nodes):
        w = str2weight["softmax_node_"+str(n)+"_w"]
        b = str2weight["softmax_node_"+str(n)+"_b"]
        inp = np.dot(w, x) + b
        node_probs.append(softmax(inp))
    print node_probs

In [0]:

x = np.random.rand(128,)

#define a variable to hold normal random values 
normal_rv = tf.Variable( tf.truncated_normal([2,3],stddev = 0.1))

#initialize the variable
init_op = tf.initialize_all_variables()

#run the graph
with tf.Session() as sess:
    sess.run(init_op) #execute init_op
    #print the random values that we sample
    print (sess.run(normal_rv))
    
for value in leaf_to_path:
    print value
    loss = []
    path, node = leaf_to_path[value]
    print path, "\n", node
    print len(node[0:])
    
    
    for n in range(4):
        print "hey"
        w = np.random.rand(2, 128)
        b = np.random.rand(2,)
        left = np.sum(w[0]*x.copy()) + b[0]
        right = np.sum(w[1]*x.copy()) + b[1]
        inp = np.array([left, right])
        print inp
        preds = softmax(inp)
        tf.Print(preds)
    
    print "\n"    
                

In [0]:
x = tf.convert_to_tensor([3,2], preferred_dtype=tf.dtypes.float32)
#x1 = K.exp(x)
y = tf.convert_to_tensor([[[2,2], [3,3]],
                          [[2,1], [4,3]]], preferred_dtype=tf.dtypes.float32)
print 'yshape', y.get_shape()
sess = tf.Session()
print 'x', sess.run(x)
#print tf.Session().run(x1)
print 'y', sess.run(y)
intermediate = K.pow(x,y)
print 'pow', sess.run(intermediate)
es = tf.reduce_prod(intermediate, axis=1)
print 'e', sess.run(es)
print sess.run(es/tf.reshape(tf.reduce_sum(es, axis=1), [2,1]))



yshape (2, 2, 2)
x [3. 2.]
y [[[2. 2.]
  [3. 3.]]

 [[2. 1.]
  [4. 3.]]]
pow [[[ 9.  4.]
  [27.  8.]]

 [[ 9.  2.]
  [81.  8.]]]
e [[243.  32.]
 [729.  16.]]
[[0.88363636 0.11636364]
 [0.9785235  0.02147651]]


In [0]:
tt = TreeTools()

        self.tree = random_binary_full_tree(list(range(output_dim)))
        self.leaf_path = {}
        for node, path in tt._get_paths(self.tree):
            if 'e' in node:
                if node.startswith('e'):
                    self.leaf_path[int(node[1:])] = path
                for l in [int(leaf) for leaf in node.split('e')]:
                    self.leaf_path[l] = path
        self.categorical = {}
for path, leaf in tt._get_leaves_paths(self.tree):
            self.categorical[leaf] = np_utils.to_categorical(path, num_classes=2)
        
self.root = self.add_weight(name="root_node",
                                    shape=(self.output_dim, len(self.tree)),
                                    initializer='uniform',
                                    trainable=True)
        self.node_vectors = {}
        for i in range(1, self.output_dim):
            self.node_vectors[i] = self.add_weight(name="node_"+str(i),
                                                   shape=(len(self.tree), len(self.tree)),
                                                   initializer='uniform',
                                                   trainable=True)
        self.leaf_weight = {}
        print self.leaf_path.keys()
        for leaf_i in self.leaf_path.keys():
            temp = []
            #print leaf_i," : ",
            for node in self.leaf_path[leaf_i]:
                #print node, self.node_vectors[node], "  ,",
                temp.append(self.node_vectors[node])
            #print ""
                
            self.leaf_weight[leaf_i] = temp
                
        
        